In [1]:
import numpy as np
import time
import tensorflow as tf

In [2]:
from eventcnn.datasources.davis import DavisDataset
from eventcnn.datasources.eventsource import EventSource, CachedDenseEventSource

In [3]:
source = CachedDenseEventSource("data/dense")

In [22]:
N_EVENT_SLICES = source.event_slices
BATCH_SIZE = 8
N_EVENT_ROWS = source.rows
N_EVENT_COLS = source.cols
LABEL_SCALE_FACTOR = np.float32(1.0 / np.max(np.abs(source.training()["labels"])))  # Rescale the positions to be in the range [-1, 1]

In [23]:
# Copied from the tensorflow tutorial in class
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def init_bias(shape):
    return tf.Variable(tf.zeros(shape))

In [24]:
def n_output_channels(layer):
    return layer.get_shape()[-1].value

def n_outputs(layer):
    out = layer.get_shape()[1].value
    for d in layer.get_shape()[2:]:
        out *= d.value
    return out


In [25]:
input_tensor = tf.placeholder(tf.float32, [None, 
                                           N_EVENT_COLS, 
                                           N_EVENT_ROWS, 
                                           N_EVENT_SLICES, 
                                           2])
dropout_keep_prob = tf.placeholder(tf.float32)

conv1 = tf.nn.conv3d(input_tensor, init_weights([3,3,3,2,6]), 
                     strides=[1,1,1,1,1], padding='SAME')
conv1 = tf.nn.relu(tf.nn.bias_add(conv1, init_bias(n_output_channels(conv1))))
pool1 = tf.nn.max_pool3d(conv1, ksize=[1, 2, 2, 2, 1], 
                         strides=[1, 2, 2, 2, 1], padding="SAME")

conv2 = tf.nn.conv3d(pool1, 
                     init_weights([3,3,2, n_output_channels(pool1),6]), 
                     strides=[1, 1, 1, 1, 1], padding="SAME")
conv2 = tf.nn.relu(tf.nn.bias_add(conv2, init_bias(n_output_channels(conv2))))
pool2 = tf.nn.max_pool3d(conv1, ksize=[1, 2, 2, 2, 1], strides=[1, 2, 2, 2, 1], padding="SAME")

fc3 = tf.reshape(pool2, [-1, n_outputs(pool2)])
fc3 = tf.add(tf.matmul(fc3, init_weights([n_outputs(pool2), 32])), init_bias(32))
fc3 = tf.nn.relu(fc3)
fc3 = tf.nn.dropout(fc3, dropout_keep_prob)

out = tf.add(tf.matmul(fc3, init_weights([n_outputs(fc3), 3])), init_bias(3))

y = tf.placeholder(tf.float32, [None, 3])

loss = tf.nn.l2_loss(tf.sub(out, tf.mul(LABEL_SCALE_FACTOR, y)))
avg_loss = tf.reduce_mean(loss)
train_optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

init = tf.initialize_all_variables()

In [26]:
TRAINING_STEPS = 10

sess = tf.Session()
sess.run(init)

for step in range(TRAINING_STEPS):
    for i in range(0, source.num_training - BATCH_SIZE, BATCH_SIZE):
        print(step, i)
        events = source.training()["events"][i:(i+BATCH_SIZE),:,...,:]
        labels = source.training()["labels"][i:(i+BATCH_SIZE),:]
        sess.run(train_optimizer, feed_dict={input_tensor: events,
                                             y: labels,
                                             dropout_keep_prob: 0.5})

    events_test = source.testing()["events"]
    labels_test = source.testing()["labels"]
    avg = sess.run(avg_loss, feed_dict={input_tensor: events_test,
                                        y: labels_test,
                                        dropout_keep_prob: 1.0})
    print("avg:", avg)
avg

0 0
0 8
0 16
0 24
0 32
0 40
0 48
0 56
0 64
0 72
0 80
0 88
0 96
0 104
0 112
0 120
0 128
0 136
0 144
0 152
0 160
0 168
0 176
0 184
0 192
0 200
0 208
0 216
0 224
0 232
0 240
0 248
0 256
0 264
0 272
0 280
0 288
0 296
0 304
0 312
0 320
0 328
0 336
0 344
0 352
0 360
0 368
0 376
0 384
0 392
0 400
0 408
0 416
0 424
0 432
0 440
0 448
0 456
0 464
0 472
0 480
0 488
0 496
0 504
0 512
0 520
0 528
0 536
0 544
0 552
0 560
0 568
0 576
0 584
0 592
0 600
0 608
0 616
0 624
0 632
0 640
0 648
avg: 12.3849
1 0
1 8
1 16
1 24
1 32
1 40
1 48
1 56
1 64
1 72
1 80
1 88
1 96
1 104
1 112
1 120
1 128
1 136
1 144
1 152
1 160
1 168
1 176
1 184
1 192
1 200
1 208
1 216
1 224
1 232
1 240
1 248
1 256
1 264
1 272
1 280
1 288
1 296
1 304
1 312
1 320
1 328
1 336
1 344
1 352
1 360
1 368
1 376
1 384
1 392
1 400
1 408
1 416
1 424
1 432
1 440
1 448
1 456
1 464
1 472
1 480
1 488
1 496
1 504
1 512
1 520
1 528
1 536
1 544
1 552
1 560
1 568
1 576
1 584
1 592
1 600
1 608
1 616
1 624
1 632
1 640
1 648
avg: 10.6835
2 0
2 8
2 16
2 24
2 

2.3358891

In [33]:
for i in range(100, 110):
    predicted = sess.run(out, feed_dict={input_tensor: events_test[i:i+1,:,...,:],
                              dropout_keep_prob: 1.0})
    expected = labels_test[i,:] * LABEL_SCALE_FACTOR
    print(predicted, expected)
    
    

[[-0.18716693 -0.14502968 -0.1650719 ]] [-0.12893723 -0.17657488 -0.23264527]
[[-0.14262208 -0.16349858 -0.12043285]] [-0.11396603 -0.18117745 -0.21677621]
[[-0.04544632 -0.11530989 -0.11746905]] [-0.1094861  -0.17524899 -0.21626075]
[[-0.03075144 -0.09589148 -0.13925767]] [-0.10909313 -0.15662379 -0.18436755]
[[-0.03155366 -0.10638125 -0.09571987]] [-0.10863434 -0.15778517 -0.16973676]
[[ 0.00344877 -0.07023229 -0.01811689]] [-0.12592036 -0.16390473 -0.16941369]
[[-0.03829715 -0.1265775  -0.137952  ]] [-0.13915086 -0.16850698 -0.16569153]
[[-0.13012195 -0.17130461 -0.09890939]] [-0.14786366 -0.18020282 -0.14778977]
[[-0.05588983 -0.09440403 -0.05762909]] [-0.16123768 -0.18644556 -0.13090448]
[[-0.06689512 -0.16188392 -0.07853331]] [-0.19940725 -0.23147917 -0.10130116]
